## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
import openpathsampling as paths
from openpathsampling.storage import Storage, AnalysisStorage
import openmmtools as omt
import simtk.openmm as omm
import simtk.unit as u
import mdtraj as md

Create Alanine in Vacuum and run it using OPS.

In [2]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [3]:
#! skip
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_positions': simtk.unit.quantity.Quantity,
 '_system': simtk.openmm.openmm.System,
 '_topology': simtk.openmm.app.topology.Topology}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [4]:
template = paths.tools.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [5]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [6]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [7]:
engine = paths.OpenMMEngine(template, system, integrator)

Let's run a simulation of 1000 steps.

In [8]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(1000).can_append])

In [9]:
print traj

Trajectory[1000]


In [10]:
#! skip
print traj[100].coordinates

[[ 0.12540653  0.28724945 -0.03791606]
 [ 0.19224785  0.21338831  0.00633061]
 [ 0.18307354  0.11970618 -0.04862791]
 [ 0.16753408  0.19555856  0.11098398]
 [ 0.33617839  0.26580858  0.00217126]
 [ 0.42706954  0.18777709  0.02430194]
 [ 0.36259776  0.3934491  -0.02740861]
 [ 0.28340924  0.45516092 -0.03844646]
 [ 0.49518085  0.45895573 -0.02352729]
 [ 0.54674739  0.42150235  0.06489863]
 [ 0.58240449  0.42757654 -0.14769138]
 [ 0.53525186  0.4694545  -0.2365952 ]
 [ 0.68233162  0.46976769 -0.13694504]
 [ 0.59507066  0.3198933  -0.15887028]
 [ 0.47329748  0.60975987 -0.00266679]
 [ 0.35901344  0.65167755  0.01499871]
 [ 0.58086395  0.68973106 -0.00448704]
 [ 0.67041749  0.6476422  -0.02473027]
 [ 0.5810535   0.83291805  0.02910487]
 [ 0.51109415  0.85531294  0.10963555]
 [ 0.67947435  0.86626923  0.06199897]
 [ 0.55347472  0.89072293 -0.05909439]] nm


In [11]:
psi = md.compute_psi(traj.md())

In [12]:
#! ignore
psi[1][0:10]

array([[ 3.14159131],
       [-3.12126136],
       [ 3.1028285 ],
       [ 3.10614586],
       [-3.09586358],
       [ 3.13675618],
       [ 3.12832665],
       [ 3.09317851],
       [ 3.08085537],
       [ 3.04846501]], dtype=float32)

In [13]:
st = Storage('engine_store_test.nc', mode='w', template=template)

In [14]:
st.engines.save(engine);

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [15]:
#! skip
st.variables['engines_json'][0][0:256] + '...'

u'{"_cls": "OpenMMEngine", "_dict": {"integrator_xml": "<?xml version=\\"1.0\\" ?>\\n<Integrator constraintTolerance=\\"1e-05\\" stepSize=\\".002\\" type=\\"VerletIntegrator\\" version=\\"1\\"/>\\n", "options": {"timestep": null, "n_frames_max": 5000, "platform": "OpenC...'

In [16]:
st.close()

### try to load the stored engine

In [17]:
st = AnalysisStorage('engine_store_test.nc')

In [18]:
engine = st.engines[0]

Run another 1000 steps to test

In [19]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(1000).can_append])

In [20]:
print traj

Trajectory[1000]
